In [3]:
from zipfile import ZipFile
import csv
import cv2
import pickle
import numpy as np
import pandas as pd
from tensorflow.keras import models, layers

In [4]:
log = pd.read_csv('./data/driving_log.csv')  
log.drop(columns=['throttle','brake','speed'],inplace=True)
log.head()


,center,left,right,steering
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0


In [21]:
def flip(images,labels):
    result_images =[]
    result_measures =[]
    for i,img in enumerate(images):
        flip_image = np.fliplr(img)
        result_images.append(flip_image)
        flip_measure = -labels[i]
        result_measures.append(flip_measure)
    return np.asarray(result_images),np.asarray(result_measures)

def image_generator(logs, batch_size, mode="train"):
    while True:
        start = 0
        end = batch_size
        while start  < len(logs):
            selected = logs[start:end]
            images,labels = batch_loader(selected)
            yield images,labels
            start += batch_size
            end += batch_size

def batch_loader(log_select):
    images = []
    measurements = []
    for l in log_select.itertuples():
        path = './data/'+l.center
        image = cv2.imread(path)
        if image is not None :
            # normal image
            images.append(image)
            measure = l.steering
            measurements.append(measure)
        else:
            print('Image not read',path)
    return np.asarray(images),np.asarray(measurements)

In [20]:
images,labels = batch_loader(log[1:2])
print(images.shape,labels.shape)
generator = image_generator(log[1:10],10)
images,labels = next(generator)
print(images.shape,labels.shape)

(1, 160, 320, 3) (1,)
(9, 160, 320, 3) (9,)


In [23]:
img_shape = (160, 320, 3)

In [29]:
model = models.Sequential()
model.add(layers.Convolution2D(6,5,5,activation='relu',input_shape=img_shape))
model.add(layers.MaxPooling2D())
model.add(layers.Dropout(0.25))
model.add(layers.Convolution2D(6,5,5,activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(120))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(84))
model.add(layers.Dense(1))
model.compile(loss='mse', optimizer='adam',metrics=['accuracy'])

In [30]:
history = model.fit(image_generator(log[:100],50),
        shuffle=True,
        steps_per_epoch=50,
        epochs=2,
        verbose=1)


Epoch 1/2
50/50 [==============================] - 3s 65ms/step - loss: 797.3839 - accuracy: 0.3756
Epoch 2/2
50/50 [==============================] - 3s 64ms/step - loss: 62.5958 - accuracy: 0.4036


In [18]:
model.save('model.h5')
